# 多様な制約を一般化した強化学習のフレームワーク

参考：
* [Safe Exploration in Reinforcement Learning: A Generalized Formulation and Algorithms](https://arxiv.org/abs/2310.03225)
* [A Survey of Constraint Formulations in Safe Reinforcement Learning](https://arxiv.org/abs/2402.02025)

CMDPでは期待値の意味での制約を考えますが，それ以外にも制約の実現方法はあります．
今回はいろいろなタイプのSafe RLを見てみましょう．

表記：
* 価値：$V_{r, h}^\pi(s):=\mathbb{E}_\pi\left[\sum_{h^{\prime}=h}^H \gamma_r^{h^{\prime}} r\left(s_{h^{\prime}}, a_{h^{\prime}}\right) \mid s_h=s\right]$
* 初期状態での期待値：$V_c^\pi(\rho):=\mathbb{E}_{s \sim \rho}\left[V_{c, 0}^\pi(s)\right]$
* $V_{\max }:=\frac{1-\gamma_r^H}{1-\gamma_r}$
* ベルマン最適作用素：$\mathcal{T}_h(Q)(s, a):=\mathbb{E}\left[r\left(s_h, a_h\right)+\gamma_r V_Q\left(s_{h+1}\right) \mid s_h=s, a_h=a\right]$, where $V_Q(s):=\max _{a \in \mathcal{A}} Q(s, a)$

また，方策空間は決定的であることを仮定します（解けるのか？）

## いろいろなSafe RL

---

**1. 期待値制約**

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad V_c^\pi(\rho) \leq \xi \text {. }
$$

これは扱いやすいのでよく使われます．
特にラグランジュ形式にすると報酬とコストが合体できるので解きやすいです．

---

**2. Safe RL with state constraints**

* [Recovery RL: Safe Reinforcement Learning with Learned Recovery Zones](https://arxiv.org/abs/2010.15920)など

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{E}_\pi\left[\sum_{h=0}^H \gamma_c^h \mathbb{I}\left(s_h \in S_{\text {unsafe }}\right)\right] \leq \xi
$$

危険な状態に突入してしまうことを避ける問題設定です．
ロボティクスの応用などでよく出てくるっぽいですね．
ちなみにこの問題は$c(s, a):=\mathbb{I}\left(s \in S_{\text {unsafe }}\right)$とすれば，**1**に含まれます．

---

**3. Joint chance constraint**

* [Safe Reinforcement Learning with Chance-constrained Model Predictive Control](https://arxiv.org/abs/2112.13941)

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{P}_\pi\left[\bigvee_{h=0}^H s_h \in S_{\text {unsafe }}\right] \leq \xi,
$$

$h=0$から$H$までを総合して，危険な状態にいる確率が一定値以下であることを保証します．
制御の問題でたまに出てくるみたいです．
これは実際解きづらいので，次のゆるい問題を解くことがあります．

$$
\mathbb{P}_\pi\left[\bigvee_{h=1}^H s_h \in S_{\text {unsafe }}\right] \leq \mathbb{E}_\pi\left[\sum_{h=1}^H \mathbb{I}\left(s_h \in S_{\text {unsafe }}\right)\right]
$$

この意味で，**2**は**3**よりもゆるい問題です．

---

**4. Expected Instantaneous Safety Constraint
with Time-variant Threshold**

* [OptLayer - Practical Constrained Optimization for Deep Reinforcement Learning in the Real World](https://arxiv.org/abs/1709.07643)など

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{E}_\pi\left[c\left(s_h, a_h\right)\right] \leq \xi_h, \forall h \in[H]
$$

あんまり見ないですが，各ステップで制約を考える設定です．
多分普通の問題よりは解きやすいのかも？
また，
$$
\eta_h:=\gamma_c^{-h} \cdot\left(\xi-\sum_{h^{\prime}=0}^{h-1} \gamma_c^{h^{\prime}} c\left(s_{h^{\prime}}, a_{h^{\prime}}\right)\right), \forall h \in[H]
$$
を使って$\mathbb{E}_\pi\left[c\left(s_h, a_h\right) \leq \eta_h\right]$を考えると，これは**1**と等価っぽいです．

**TODO: これは$\eta_h$が方策に依存してしまうので，等価ではないのでは？各エピソードで方策が変わるとすると，これはNon-stationary MDPを解くことになるかもしれない．**

---
**5. Almost surely safe RL**

* [Saute RL: Almost Surely Safe Reinforcement Learning Using State Augmentation](https://arxiv.org/abs/2202.06558)で導入されたっぽい

$$
\max _\pi V_r^\pi(\rho) \quad \text { subject to } \quad \mathbb{P}_\pi \left[\sum_{h=1}^H \gamma_c^h c\left(s_h, a_h\right) \leq \xi \right]=1,
$$

これは期待値よりも強い制約を課す問題です．
TODO: これNP困難になりそうだけど大丈夫か？Mean-variance MDPを考えるとこれNP困難になりそうな気がする．

---

**6. Almost Surely Instantaneous Safety
Constraint with Time-invariant Threshold**

* [Safe Exploration in Reinforcement Learning: A Generalized Formulation and Algorithms](https://arxiv.org/abs/2310.03225)
* [End-to-End Safe Reinforcement Learning through Barrier Functions for Safety-Critical Continuous Control Tasks](https://arxiv.org/abs/1903.08792)

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{P}_\pi\left[c\left(s_h, a_h\right) \leq \xi\right]=1, \forall h \in[H]
$$

各ステップ全部で制約を満たすことを考え，しかもAlmost surelyであることを要求します．
制御工学におけるバリア関数やリアプノフ関数とかで使われるっぽい？

---

**7. Almost Surely Instantaneous Safety Constraint with Time-variant Threshold**

* [Safe Exploration in Reinforcement Learning: A Generalized Formulation and Algorithms](https://arxiv.org/abs/2310.03225)

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \quad \mathbb{P}_\pi\left[c\left(s_h, a_h\right) \leq \xi_h\right]=1, \forall h \in[H]
$$

これは複数の問題を一般化できる点で便利っぽい．
* 明らかにこれは**6**の一般化
* また，
$$
\eta_h:=\gamma_c^{-h} \cdot\left(\xi-\sum_{h^{\prime}=0}^{h-1} \gamma_c^{h^{\prime}} c\left(s_{h^{\prime}}, a_{h^{\prime}}\right)\right), \forall h \in[H]
$$
を使って$\mathbb{P}_\pi\left[c\left(s_h, a_h\right) \leq \eta_h\right]=1$を考えると，これは**5**と等価っぽいです．

**TODO: これは$\eta_h$が方策に依存してしまうので，等価ではないのでは？各エピソードで方策が変わるとすると，これはNon-stationary MDPを解くことになるかもしれない．**

---

**8. 他の問題**

* [Policy Gradients with Variance Related Risk Criteria](https://arxiv.org/abs/1206.6404)

$$
\max _\pi V_r^\pi(\rho) \quad \text { s.t. } \operatorname{Var}\left[V_r^\pi(\rho)\right]<\xi \text {. }
$$

分散が一定以下になるような制約を課す問題です．これはMean-variance MDPみたいにNP困難になりそうな気がする．

後は自然言語を利用するアプローチもあるみたいです．

TODO: Shieldingの話
